In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import Preprocess_gokhanEr as pp

In [3]:
import os
import sys

## TF-IDF and Sparse Matrix

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

- lower case
- contraction to expansion
- remove emails, urls, special chars, mentions, hashtags, html tags, 
- remove more than 2 repeated chars. youuu- you

In [5]:
df = pd.read_csv("twitter30k_cleaned.csv")
df.head()

,twitts,sentiment
0,robbiebronniman sounds like a great night,1
1,damn the person who stolde my wallet may karma...,1
2,greetings from the piano bench photo,1
3,drewryanscott i love it i love you haha forget...,1
4,kissthestars pretty pretty pretty please pakid...,0


In [6]:
df["sentiment"].value_counts()

1    15000
0    15000
Name: sentiment, dtype: int64

In [7]:
X = df["twitts"]
y = df["sentiment"]

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(X)

In [10]:
pd.DataFrame(data = X.toarray(), columns = tfidf.get_feature_names() ).head()

,005603,01,010,0100,02,0200,0230,0410,0430,05,...,ñðñðµ,ñðñð½ññ,ññðµð²ð,ññðºð,ññð¾,ññð¾ðñ,ø³ùùû,øµøøª,øø,ùù
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
X

<30000x42126 sparse matrix of type '<class 'numpy.float64'>'
	with 350520 stored elements in Compressed Sparse Row format>

In [12]:
# % kaçı 0 temsil ediyor bütün verilerden 

350520 / (X.shape[0]*X.shape[1])

0.00027735840098751366

In [13]:
# Burda 0 olmayan değerler !!!

X.toarray().astype("bool").sum().sum()

350520

In [14]:
# Kolon sayıları !!

len(tfidf.vocabulary_)

42126

In [15]:
print("X Shape : {}".format(X.shape))
print("Y Shape : {}".format(y.shape))

X Shape : (30000, 42126)
Y Shape : (30000,)


In [16]:
type(X),type(y)

(scipy.sparse.csr.csr_matrix, pandas.core.series.Series)

In [17]:
sys.getsizeof(X)

64

In [18]:
len(X.data)

350520

In [19]:
d = (X.data.nbytes + X.indptr.nbytes + X.indices.nbytes)
d

4326244

In [20]:
d / 2 ** 20

4.125827789306641

In [21]:
(X.shape[0] * X.shape[1])*8 / (2**20)

9641.876220703125

## Non - Negative Matrix Factorization (NMF)

In [22]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, stratify=y, random_state=0)

In [23]:
from sklearn.decomposition import PCA, NMF

In [24]:
from warnings import filterwarnings
filterwarnings("ignore")

In [25]:
%%time
nmf = NMF(n_components=50, random_state= 0 )
X_train_nmf = nmf.fit_transform(X_train)

print("X Train Shape : {}".format(X_train.shape))
print("NMF X Train Shape : {}".format(X_train_nmf.shape))

X Train Shape : (24000, 42126)
NMF X Train Shape : (24000, 50)
CPU times: user 1min 49s, sys: 25.4 s, total: 2min 14s
Wall time: 19.2 s


In [26]:
def run_svm(clf, X_train,X_test, y_train, y_test):
    
    from sklearn.metrics import classification_report
    
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    
    print()
    print("Printing Report")
    print(classification_report(y_test,y_pred))

In [27]:
%%time

clf = LinearSVC()

X_test_nmf = nmf.transform(X_test)

run_svm(clf, X_train_nmf, X_test_nmf, y_train, y_test)


Printing Report
              precision    recall  f1-score   support

           0       0.66      0.62      0.64      3000
           1       0.64      0.68      0.66      3000

    accuracy                           0.65      6000
   macro avg       0.65      0.65      0.65      6000
weighted avg       0.65      0.65      0.65      6000

CPU times: user 766 ms, sys: 171 ms, total: 937 ms
Wall time: 135 ms


In [28]:
%%time
clf = LinearSVC()
run_svm(clf, X_train, X_test, y_train, y_test)


Printing Report
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      3000
           1       0.75      0.75      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000

CPU times: user 316 ms, sys: 95.9 ms, total: 412 ms
Wall time: 68.5 ms


## Trancated Singular Value Decomposition (TSVD)

In [29]:
from sklearn.decomposition import TruncatedSVD as TSVD

In [30]:
%%time
tsvd = TSVD(n_components=500, random_state=0)
X_train_tsvd = tsvd.fit_transform(X_train)

CPU times: user 37.6 s, sys: 5.89 s, total: 43.5 s
Wall time: 8.1 s


In [31]:
%%time
X_test_tsvd = tsvd.transform(X_test)

CPU times: user 30.9 ms, sys: 35.8 ms, total: 66.7 ms
Wall time: 65.9 ms


In [32]:
%%time
clf = LinearSVC()
run_svm(clf,X_train_tsvd, X_test_tsvd, y_train, y_test)


Printing Report
              precision    recall  f1-score   support

           0       0.75      0.72      0.73      3000
           1       0.73      0.76      0.74      3000

    accuracy                           0.74      6000
   macro avg       0.74      0.74      0.74      6000
weighted avg       0.74      0.74      0.74      6000

CPU times: user 1.25 s, sys: 48.1 ms, total: 1.3 s
Wall time: 1.27 s


## GridSearch For Logistic Regression HyperParameters Tuning

In [33]:
import numpy as np
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

In [34]:
train = X_train_tsvd.copy()
test = X_test_tsvd.copy()

In [35]:
model = LogisticRegression(solver = "liblinear")

In [36]:
penalty = ["l1","l2"]
C = np.logspace(0,4,10)
max_iter = [100,500]

In [37]:
hyperparameters = dict(penalty = penalty, C = C, max_iter = max_iter)
hyperparameters

{'penalty': ['l1', 'l2'],
 'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
        5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
        3.59381366e+03, 1.00000000e+04]),
 'max_iter': [100, 500]}

In [38]:
best_grid = GridSearchCV(estimator=model,
                         param_grid= hyperparameters,
                         n_jobs=-1,
                         verbose=0,
                         cv = 5,
                         scoring= "accuracy").fit(train, y_train)

In [39]:
best_grid.best_params_

{'C': 2.7825594022071245, 'max_iter': 100, 'penalty': 'l2'}

In [40]:
best_grid.best_estimator_

LogisticRegression(C=2.7825594022071245, solver='liblinear')

In [41]:
y_pred = best_grid.predict(test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.74      0.73      0.73      3000
           1       0.73      0.75      0.74      3000

    accuracy                           0.74      6000
   macro avg       0.74      0.74      0.74      6000
weighted avg       0.74      0.74      0.74      6000



In [42]:
best_grid.cv_results_

{'mean_fit_time': array([2.27310109, 1.68912268, 2.17653861, 1.72087698, 2.59303317,
        2.26724677, 2.39183235, 2.29484634, 2.43945127, 2.71879902,
        2.79291348, 2.79515843, 2.59631777, 2.95721641, 2.59473624,
        2.90127726, 2.59772992, 3.18376508, 2.80932779, 3.16676898,
        2.93237042, 3.16134315, 2.63916216, 3.0712461 , 2.72243242,
        3.36271706, 2.93576059, 3.36351504, 2.79816551, 3.3838192 ,
        2.75814419, 3.331811  , 2.70543404, 3.35479722, 2.96279202,
        3.31807675, 2.86026697, 3.40303712, 2.83250399, 3.0511848 ]),
 'std_fit_time': array([0.12855655, 0.07715574, 0.16191524, 0.01529905, 0.22569754,
        0.17452495, 0.19169113, 0.15136724, 0.14879996, 0.09770924,
        0.19063956, 0.0612842 , 0.17085343, 0.12111569, 0.25469827,
        0.11044306, 0.14550115, 0.07253506, 0.29501206, 0.13695923,
        0.16045726, 0.14078523, 0.26720667, 0.15497506, 0.33080503,
        0.19186775, 0.21609552, 0.21005908, 0.15355441, 0.21163235,
        0.252

In [43]:
best_grid.cv_results_["mean_test_score"]

array([0.746875  , 0.7475    , 0.74691667, 0.7475    , 0.74645833,
       0.74770833, 0.7465    , 0.74770833, 0.74704167, 0.74729167,
       0.747125  , 0.74729167, 0.74708333, 0.74745833, 0.74704167,
       0.74745833, 0.74691667, 0.747125  , 0.74691667, 0.747125  ,
       0.74691667, 0.747     , 0.746875  , 0.747     , 0.74691667,
       0.74695833, 0.74691667, 0.74695833, 0.74691667, 0.74691667,
       0.74695833, 0.74691667, 0.74691667, 0.74691667, 0.74691667,
       0.74691667, 0.74691667, 0.74691667, 0.74691667, 0.74691667])

In [44]:
best_grid.cv_results_["params"]

[{'C': 1.0, 'max_iter': 100, 'penalty': 'l1'},
 {'C': 1.0, 'max_iter': 100, 'penalty': 'l2'},
 {'C': 1.0, 'max_iter': 500, 'penalty': 'l1'},
 {'C': 1.0, 'max_iter': 500, 'penalty': 'l2'},
 {'C': 2.7825594022071245, 'max_iter': 100, 'penalty': 'l1'},
 {'C': 2.7825594022071245, 'max_iter': 100, 'penalty': 'l2'},
 {'C': 2.7825594022071245, 'max_iter': 500, 'penalty': 'l1'},
 {'C': 2.7825594022071245, 'max_iter': 500, 'penalty': 'l2'},
 {'C': 7.742636826811269, 'max_iter': 100, 'penalty': 'l1'},
 {'C': 7.742636826811269, 'max_iter': 100, 'penalty': 'l2'},
 {'C': 7.742636826811269, 'max_iter': 500, 'penalty': 'l1'},
 {'C': 7.742636826811269, 'max_iter': 500, 'penalty': 'l2'},
 {'C': 21.544346900318832, 'max_iter': 100, 'penalty': 'l1'},
 {'C': 21.544346900318832, 'max_iter': 100, 'penalty': 'l2'},
 {'C': 21.544346900318832, 'max_iter': 500, 'penalty': 'l1'},
 {'C': 21.544346900318832, 'max_iter': 500, 'penalty': 'l2'},
 {'C': 59.94842503189409, 'max_iter': 100, 'penalty': 'l1'},
 {'C': 59.9

## GridSearch For SVM HyperParameters Tuning

In [45]:
from sklearn.svm import SVC

In [46]:
hyperparameters = [ 
     {"kernel" : ["rbf"],
      "gamma" : [1e-1,1e-2,1e-3,1e-4,1e-5],
      "C" : [1, 10, 100, 500, 1000],
      "degree" : [2,3,4,5]},

     {"kernel" : ["linear"],
      "C" : [1, 10, 100, 500, 1000]}]

In [47]:
hyperparameters

[{'kernel': ['rbf'],
  'gamma': [0.1, 0.01, 0.001, 0.0001, 1e-05],
  'C': [1, 10, 100, 500, 1000],
  'degree': [2, 3, 4, 5]},
 {'kernel': ['linear'], 'C': [1, 10, 100, 500, 1000]}]

In [48]:
from sklearn import datasets


iris= datasets.load_iris()
X = iris.data
y = iris.target

In [49]:
X_train,X_test,y_train,y_test = train_test_split(X,y , random_state=0,stratify=y,test_size=0.2)

In [50]:
scores = ["precision","recall"]

def run_tuning(model,X_train,X_test,y_train,y_test, hyperparameters, scores, cv = 5):
    
    for score in scores:
        
        print("Tuning HyperParamters For %s" % score.upper())
        print()
        
        best_grid = GridSearchCV(estimator=model, 
                                 param_grid=hyperparameters, 
                                 n_jobs=-1,
                                 verbose=0,
                                 cv = cv,
                                 scoring="%s_macro" % score).fit(X_train,y_train)
        
        print("Best Parameters Set Found: ",end = "\n\n" )
        print(best_grid.best_params_)
        
        print("Grid Scores In Process : ",end = "\n\n")
        means = best_grid.cv_results_["mean_test_score"]
        
        for mean, params in zip(means,best_grid.cv_results_["params"]):
            print("%0.3f for %r" % (mean,params))
        
        print("",end = "\n\n")
            
        print("Detailed Classification Report")
        y_pred = best_grid.predict(X_test)
        print(classification_report(y_test,y_pred))

In [51]:
run_tuning(SVC(), X_train,X_test,y_train,y_test,hyperparameters,scores,cv= 5)

Tuning HyperParamters For PRECISION

Best Parameters Set Found: 

{'C': 1, 'kernel': 'linear'}
Grid Scores In Process : 

0.957 for {'C': 1, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'}
0.920 for {'C': 1, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}
0.925 for {'C': 1, 'degree': 2, 'gamma': 0.001, 'kernel': 'rbf'}
0.925 for {'C': 1, 'degree': 2, 'gamma': 0.0001, 'kernel': 'rbf'}
0.925 for {'C': 1, 'degree': 2, 'gamma': 1e-05, 'kernel': 'rbf'}
0.957 for {'C': 1, 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf'}
0.920 for {'C': 1, 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf'}
0.925 for {'C': 1, 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf'}
0.925 for {'C': 1, 'degree': 3, 'gamma': 0.0001, 'kernel': 'rbf'}
0.925 for {'C': 1, 'degree': 3, 'gamma': 1e-05, 'kernel': 'rbf'}
0.957 for {'C': 1, 'degree': 4, 'gamma': 0.1, 'kernel': 'rbf'}
0.920 for {'C': 1, 'degree': 4, 'gamma': 0.01, 'kernel': 'rbf'}
0.925 for {'C': 1, 'degree': 4, 'gamma': 0.001, 'kernel': 'rbf'}
0.925 for {'C': 1, 'degree': 4, 'gamma':

## GridSearch For RandomForesClassifier  HyperParameters Tuning 

In [52]:
from sklearn.ensemble import RandomForestClassifier

hyperparameters = {
    "bootstrap" : [True],
    "max_depth" : [10,100],
    "max_features" : [2,3, X.shape[1]],
    "min_samples_leaf" : [2,5],
    "min_samples_split" : [1, 5, 10],
    "n_estimators" : [10, 100, 200] 
    
}

In [53]:
run_tuning(RandomForestClassifier(), X_train,X_test,y_train,y_test,hyperparameters,scores,cv= 5)

Tuning HyperParamters For PRECISION

Best Parameters Set Found: 

{'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 10}
Grid Scores In Process : 

nan for {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 1, 'n_estimators': 10}
nan for {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 1, 'n_estimators': 100}
nan for {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 1, 'n_estimators': 200}
0.944 for {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 10}
0.952 for {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
0.952 for {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estim

Best Parameters Set Found: 

{'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 10}
Grid Scores In Process : 

nan for {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 1, 'n_estimators': 10}
nan for {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 1, 'n_estimators': 100}
nan for {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 1, 'n_estimators': 200}
0.950 for {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 10}
0.950 for {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
0.950 for {'bootstrap': True, 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
0.958 for {'bootstrap': 

## Random Search For Best Parameter Selection

In [54]:
from scipy.stats import uniform, loguniform
from sklearn.model_selection import RandomizedSearchCV

In [55]:
C = uniform(loc=0, scale = 4)
gamma = loguniform(1e-5,1e-1)

hyperparameters = dict( C = C, gamma = gamma)
hyperparameters

{'C': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fb4a8f5ed10>,
 'gamma': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fb4a8f5e0d0>}

In [56]:
clf = RandomizedSearchCV(SVC(), hyperparameters, cv = 5, n_jobs=-1).fit(X_train,y_train)

In [57]:
clf.best_params_

{'C': 3.1817000481656224, 'gamma': 0.03900862190184619}

In [58]:
clf.best_score_

0.95

In [59]:
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



## Selecting Best Models From Multiple ML Algorithms

In [60]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [61]:
np.random.seed(0)

In [62]:
pipe = Pipeline([("classifier",RandomForestClassifier())])
pipe

Pipeline(steps=[('classifier', RandomForestClassifier())])

In [63]:
search_space = [{"classifier" : [LogisticRegression(solver = "liblinear")],
                 "classifier__penalty" : ["l1","l2"],
                 "classifier__C" : np.logspace(0,4,10)},
                 
                {"classifier" : [RandomForestClassifier(n_jobs=-1)],
                 "classifier__n_estimators" : [10,100],
                 "classifier__max_features" : [1,3]},
                
               
                {"classifier" : [SVC()],
                 "classifier__C" : [1,5],
                 "classifier__gamma" : [1e-1, 1e-2]}]

In [64]:
best_grid = GridSearchCV(pipe, search_space,cv = 5, n_jobs=-1, verbose=0).fit(X_train,y_train)

In [65]:
best_grid.best_estimator_

Pipeline(steps=[('classifier', SVC(C=5, gamma=0.1))])

In [66]:
best_grid.best_params_

{'classifier': SVC(C=5, gamma=0.1),
 'classifier__C': 5,
 'classifier__gamma': 0.1}

In [67]:
pipe

Pipeline(steps=[('classifier', RandomForestClassifier())])